In [ ]:
!pip install trimesh
import os
import glob
import trimesh
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from matplotlib import pyplot as plt

tf.random.set_seed(1234)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 693.9/693.9 kB 7.4 MB/s eta 0:00:00


In [ ]:
# Use hardware accelerator for training
#physical_devices = tf.config.experimental.list_physical_devices('GPU')
#print("GPUs Available: ", len(physical_devices))
#tf.config.experimental.set_memory_growth(physical_devices[0], True)

## Download the Dataset

In [ ]:
DATA_DIR = tf.keras.utils.get_file(
    "modelnet.zip",
    "http://3dvision.princeton.edu/projects/2014/3DShapeNets/ModelNet10.zip",
    extract=True,
)
DATA_DIR = os.path.join(os.path.dirname(DATA_DIR), "ModelNet10")

## Visualize a Mesh from the Dataset


In [ ]:
mesh = trimesh.load(os.path.join(DATA_DIR, "bed/train/bed_0001.off"))
mesh.show()

## Sample a Mesh and Show Result

In [ ]:
!pip install rtree

In [ ]:
points_test=trimesh.sample.sample_surface(mesh, count=2048)
faces=points_test[1]
print(faces)
points_test=points_test[0]

fig = plt.figure(figsize=(5, 5))
ax = fig.add_subplot(111, projection="3d")
ax.scatter(points_test[:, 0], points_test[:, 1], points_test[:, 2])
ax.set_axis_off()
plt.show()


#Function to Parse Data


In [ ]:
def parse_dataset(num_points=2048):

    train_points = []
    train_labels = []
    test_points = []
    test_labels = []
    class_map = {}
    folders = glob.glob(os.path.join(DATA_DIR, "[!README]*"))

    for i, folder in enumerate(folders):
        print("processing class: {}".format(os.path.basename(folder)))
        # store folder name with ID so we can retrieve later
        class_map[i] = folder.split("/")[-1]
        # gather all files
        train_files = glob.glob(os.path.join(folder, "train/*"))
        test_files = glob.glob(os.path.join(folder, "test/*"))

        for f in train_files:
            train_points.append(trimesh.load(f).sample(num_points))
            train_labels.append(i)

        for f in test_files:
            test_points.append(trimesh.load(f).sample(num_points))
            test_labels.append(i)

    return (
        np.array(train_points),
        np.array(test_points),
        np.array(train_labels),
        np.array(test_labels),
        class_map,
    )

#Parse the Dataset and Process each Class


In [ ]:
NUM_POINTS = 2048
NUM_CLASSES = 10
BATCH_SIZE = 32

train_points, test_points, train_labels, test_labels, CLASS_MAP = parse_dataset(NUM_POINTS)

#Augenmentation for Train Dataset


In [ ]:
def augment(points, label):
    # jitter points
    points += tf.random.uniform(points.shape, -0.005, 0.005, dtype=tf.float64)
    # shuffle points
    points = tf.random.shuffle(points)
    return points, label


train_dataset = tf.data.Dataset.from_tensor_slices((train_points, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_points, test_labels))

train_dataset = train_dataset.shuffle(len(train_points)).map(augment).batch(BATCH_SIZE)
test_dataset = test_dataset.shuffle(len(test_points)).batch(BATCH_SIZE)

In [ ]:
test_dataset_normal = tf.data.Dataset.from_tensor_slices((test_points_normal, test_labels_normal))
test_dataset_normal = test_dataset_normal.shuffle(len(test_points_normal)).batch(BATCH_SIZE)


#Functions to Build the Model


In [ ]:
def conv_bn(x, filters):
    x = layers.Conv1D(filters, kernel_size=1, padding="valid")(x)
    x = layers.BatchNormalization(momentum=0.0)(x)
    return layers.Activation("relu")(x)


def dense_bn(x, filters):
    x = layers.Dense(filters)(x)
    x = layers.BatchNormalization(momentum=0.0)(x)
    return layers.Activation("relu")(x)

In [ ]:
class OrthogonalRegularizer(keras.regularizers.Regularizer):
    def __init__(self, num_features, l2reg=0.001):
        self.num_features = num_features
        self.l2reg = l2reg
        self.eye = tf.eye(num_features)

    def __call__(self, x):
        x = tf.reshape(x, (-1, self.num_features, self.num_features))
        xxt = tf.tensordot(x, x, axes=(2, 2))
        xxt = tf.reshape(xxt, (-1, self.num_features, self.num_features))
        return tf.reduce_sum(self.l2reg * tf.square(xxt - self.eye))

#Function to create T-net Layers


In [ ]:
def tnet(inputs, num_features):

    # Initalise bias as the indentity matrix
    bias = keras.initializers.Constant(np.eye(num_features).flatten())
    reg = OrthogonalRegularizer(num_features)

    x = conv_bn(inputs, 32)
    x = conv_bn(x, 64)
    x = conv_bn(x, 512)
    x = layers.GlobalMaxPooling1D()(x)
    x = dense_bn(x, 256)
    x = dense_bn(x, 128)
    x = layers.Dense(
        num_features * num_features,
        kernel_initializer="zeros",
        bias_initializer=bias,
        activity_regularizer=reg,
    )(x)
    feat_T = layers.Reshape((num_features, num_features))(x)
    # Apply affine transformation to input features
    return layers.Dot(axes=(2, 1))([inputs, feat_T])

#Create the Convolutional Neural Network


In [ ]:
inputs = keras.Input(shape=(NUM_POINTS, 3))

x = tnet(inputs, 3)
x = conv_bn(x, 64)
x = conv_bn(x, 64)
x = tnet(x, 64)
x = conv_bn(x, 64)
x = conv_bn(x, 128)
x = conv_bn(x, 1024)
x = layers.GlobalMaxPooling1D()(x)
x = dense_bn(x, 512)
x = layers.Dropout(0.3)(x)
x = dense_bn(x, 256)
x = layers.Dropout(0.3)(x)

outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs, name="pointnet")
model.summary()

#Compile and Train the Model


In [ ]:
model.load_weights("/content/pointnet_poids.h5")


In [ ]:
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["sparse_categorical_accuracy"],
)

model.fit(train_dataset, epochs=50, validation_data=test_dataset)

In [ ]:
points = test_dataset
preds = model.predict(points)
preds = tf.math.argmax(preds, -1)

# Calculate accuracy
accuracy = np.mean((preds == labels).numpy())
print("Accuracy:", accuracy)

#  l'ajout de rotation au data

In [ ]:
print (len(train_points))


3991


# Avec distribution fixe

In [ ]:
def rotate_point_cloud_z(point_cloud, angle_degrees):
    angle_radians = np.radians(angle_degrees)
    rotation_matrix = np.array([
        [np.cos(angle_radians), -np.sin(angle_radians), 0],
        [np.sin(angle_radians), np.cos(angle_radians), 0],
        [0, 0, 1]
    ])
    rotated_point_cloud = np.dot(point_cloud, rotation_matrix.T)
    return rotated_point_cloud

In [ ]:
def augment_class_data_with_rotation(class_data, num_duplicates=1):
    augmented_data = []
    angles = [40, 80, 120, 170]  # Liste des angles de rotation
    for angle in angles:
        for _ in range(num_duplicates):
            # Rotation de chaque élément de classe
            rotated_data = rotate_point_cloud_z(class_data, angle)
            augmented_data.append(rotated_data)
    augmented_data.append(class_data)
    return np.vstack(augmented_data)

In [ ]:
def augment_dataset_with_rotation(data, labels, num_duplicates=4):
    augmented_data = []
    augmented_labels = []
    unique_labels = np.unique(labels)
    for label in unique_labels:
        class_indices = np.where(labels == label)[0]
        class_data = data[class_indices]
        for _ in range(num_duplicates):
            # Ajout de rotation à chaque élément de classe avec bruit
            augmented_class_data = augment_class_data_with_rotation(class_data)
            max_shape = max([data.shape for data in augmented_class_data])
            augmented_class_data_resized = [np.pad(data, ((0, max_shape[0] - data.shape[0]), (0, 0)), mode='constant') for data in augmented_class_data]
            augmented_data.extend(augmented_class_data_resized)
            augmented_labels.extend([label] * len(augmented_class_data_resized))
    return np.array(augmented_data), np.array(augmented_labels)

In [ ]:
# Utilisation de la fonction pour augmenter les données avec bruit et rotation
train_points_augmented_with_rotation, train_labels_augmented_with_rotation = augment_dataset_with_rotation(train_points, train_labels, num_duplicates=4)
test_points_augmented_with_rotation, test_labels_augmented_with_rotation = augment_dataset_with_rotation(test_points, test_labels, num_duplicates=4)


In [ ]:
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["sparse_categorical_accuracy"],
)

model.fit(train_points_augmented_with_rotation, epochs=30, validation_data=test_dataset)




In [ ]:
# Calcul de l'accuracy sur les données de test augmentées
points = test_points_augmented_with_rotation
labels = test_labels_augmented_with_rotation

# Run test data through model
preds = model.predict(points)
preds = tf.math.argmax(preds, -1)

# Calculate accuracy
accuracy = np.mean((preds == labels).numpy())
print("Accuracy:", accuracy)

# Avec Distribution aleatoire

In [ ]:
def rotate_point_cloud_z(point_cloud,angle_degrees):
  angle_degrees = np.random(10,360)
  rotation_matrix=np.array([
      [np.cos(angle_degrees),-np.sin(angle_degrees),0],
      [np.sin(angle_degrees),np.cos(angle_degrees),0],
      [0,0,1]
  ])

In [ ]:
def augment_class_data_with_rotation(class_data, num_duplicates=4):
    augmented_data = []
    for _ in range(num_duplicates):

        rotated_data = rotate_point_cloud_z(class_data, angle)
        augmented_data.append(rotated_data)
    augmented_data.append(class_data)
    return np.vstack(augmented_data)

In [ ]:
def augment_dataset_with_rotation(data, labels, num_duplicates=4):
    augmented_data = []
    augmented_labels = []
    unique_labels = np.unique(labels)
    for label in unique_labels:
        class_indices = np.where(labels == label)[0]
        class_data = data[class_indices]
        for _ in range(num_duplicates):
            # Ajout de rotation à chaque élément de classe avec bruit
            augmented_class_data = augment_class_data_with_rotation(class_data)
            max_shape = max([data.shape for data in augmented_class_data])
            augmented_class_data_resized = [np.pad(data, ((0, max_shape[0] - data.shape[0]), (0, 0)), mode='constant') for data in augmented_class_data]
            augmented_data.extend(augmented_class_data_resized)
            augmented_labels.extend([label] * len(augmented_class_data_resized))
    return np.array(augmented_data), np.array(augmented_labels)

In [ ]:
# Utilisation de la fonction pour augmenter les données avec bruit et rotation
train_points_augmented_with_rotation, train_labels_augmented_with_rotation = augment_dataset_with_rotation(train_points, train_labels, num_duplicates=4)
test_points_augmented_with_rotation, test_labels_augmented_with_rotation = augment_dataset_with_rotation(test_points, test_labels, num_duplicates=4)


In [ ]:
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["sparse_categorical_accuracy"],
)

model.fit(train_points_augmented_with_rotation, epochs=30, validation_data=test_dataset)




In [ ]:
# Calcul de l'accuracy sur les données de test augmentées
points = test_points_augmented_with_rotation
labels = test_labels_augmented_with_rotation

# Run test data through model
preds = model.predict(points)
preds = tf.math.argmax(preds, -1)

# Calculate accuracy
accuracy = np.mean((preds == labels).numpy())
print("Accuracy:", accuracy)

#Avec Distribution Uniforme

In [ ]:
def rotatio_z(point_cloud,a):
  a = np.random.uniform(10,360)
  rotation_matrix=np.array([
      [np.cos(a),-np.sin(a),0],
      [np.sin(a),np.cos(a),0],
      [0,0,1]
  ])

In [ ]:
def augment_class_data_with_rotation(class_data, num_duplicates=4):

  augmented_data = []
  for _ in range(num_duplicates):
    a=np.random.uniform(10,360)
    rotated_data = rotatio_z(class_data, a)
    augmented_data.append(rotated_data)
  augmented_data.append(class_data)
  return np.vstack(augmented_data)

In [ ]:
def augment_dataset_with_rotation(data, labels, num_duplicates=4):
    augmented_data = []
    augmented_labels = []
    unique_labels = np.unique(labels)
    max_shape = max([data.shape for data in data])
    for label in unique_labels:
        class_indices = np.where(labels == label)[0]
        class_data = data[class_indices]
        for _ in range(num_duplicates):
            # Ajout de rotation à chaque élément de classe avec bruit
            augmented_class_data = augment_class_data_with_rotation(class_data)
            augmented_class_data_resized = [np.pad(data, ((0, max_shape[0] - data.shape[0]), (0, 0)), mode='constant') for data in augmented_class_data]
            augmented_data.extend(augmented_class_data_resized)
            augmented_labels.extend([label] * len(augmented_class_data_resized))
    return np.array(augmented_data), np.array(augmented_labels)

# Utilisation de la fonction pour augmenter les données avec bruit et rotation
train_points_augmented_with_rotation, train_labels_augmented_with_rotation = augment_dataset_with_rotation(train_points, train_labels, num_duplicates=4)
test_points_augmented_with_rotation, test_labels_augmented_with_rotation = augment_dataset_with_rotation(test_points, test_labels, num_duplicates=4)

In [ ]:
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["sparse_categorical_accuracy"],
)

model.fit(train_points_augmented_with_rotation, epochs=30, validation_data=test_dataset)




NameError: name 'model' is not defined

In [ ]:
# Calcul de l'accuracy sur les données de test augmentées
points = test_points_augmented_with_rotation
labels = test_labels_augmented_with_rotation

# Run test data through model
preds = model.predict(points)
preds = tf.math.argmax(preds, -1)

# Calculate accuracy
accuracy = np.mean((preds == labels).numpy())
print("Accuracy:", accuracy)